In [45]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from scipy.spatial.distance import cdist

In [37]:
data = pd.read_csv('datasets/balance-scale.csv', sep=',', skiprows=1, header=0)

y = data['class']
X = data.drop(['class'], axis=1).values

le = LabelEncoder()
y = le.fit_transform(y)

X.shape, y.shape

((625, 4), (625,))

In [26]:
kmeans = KMeans(n_clusters=len(np.unique(y)))
kmeans.fit(X)
sum(kmeans.labels_==0),sum(kmeans.labels_==1),sum(kmeans.labels_==2),kmeans.labels_

(200,
 175,
 250,
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,

In [41]:
N = confusion_matrix(y, kmeans.labels_)
N # [true, predicted]

array([[ 18,   8,  23],
       [ 91,  31, 166],
       [ 91, 136,  61]])

In [52]:
# h = np.zeros(N.shape)
r = cdist(kmeans.cluster_centers_, kmeans.cluster_centers_)
r

array([[ 0.        ,  2.72752878,  2.68909836],
       [ 2.72752878,  0.        ,  2.75828992],
       [ 2.68909836,  2.75828992,  0.        ]])

In [61]:
sig = 0.75
h = np.exp(-r/sig)
h

array([[ 1.        ,  0.02633899,  0.02772379],
       [ 0.02633899,  1.        ,  0.02528055],
       [ 0.02772379,  0.02528055,  1.        ]])

In [62]:
N[0, :]

array([18,  8, 23])